<a href="https://colab.research.google.com/github/Ekliipce/Machine-Learning-for-Biomedical/blob/pre-processing/eeg/EEG_and_alcohol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Electroencephalogram (EEG) and alcohol


## **EEG**
#### **What is EEG ?**
An electroencephalogram (EEG) is a test that records the brain's electrical activity noninvasively through electrodes placed on the scalp. The procedure involves placing these electrodes that are connected by wires to a computer, which then records and analyzes the electrical impulses in the brain. EEG is used for diagnosing and managing brain-related disorders like epilepsy, monitoring brain activity during surgeries, and conducting neuroscience research.

EEG patterns, consisting of different waves, are analyzed to understand normal or abnormal brain function. The procedure is safe, though preparation is required, and it might be slightly uncomfortable. EEG primarily detects activity in the brain's cortex with limited spatial resolution and can be affected by various factors like age and medication. Unlike MRI and CT scans that visualize brain structure, EEG captures real-time activity, making it a valuable tool in neuroscience and medicine.
<br><br>
#### **What does an EEG help diagnose?**

EEG is used primarily to diagnose conditions that affect brain activity. It’s particularly useful in identifying epilepsy and other seizure disorders by capturing the electrical activity of the brain. Besides, EEG can also help diagnose or manage other conditions like sleep disorders, depth of anesthesia, coma, encephalopathies, brain death, and certain psychiatric disorders. It is often used in conjunction with other diagnostic tools to provide comprehensive insights into brain health and function.
<br><br>

#### **What factors can influence the results of an EEG?**

Various factors can influence EEG results. Medications (such as sedatives, anti-epileptic drugs) can alter electrical activity in the brain, affecting the test's findings. The patient's age and overall brain development can also play a role in the results. The physical and mental state of the patient during the test, like being stressed, relaxed, asleep, or awake, can also influence the brain's electrical activity. External interference from electronic devices and not following preparatory instructions (like washing hair to ensure good electrode contact) can also impact the data quality and test outcomes.
<br><br>
#### **How reliable is EEG in diagnosing various brain disorders?**

EEG is a reliable tool for diagnosing disorders related to abnormal brain activity, like epilepsy. However, its reliability can be influenced by the technician's skill, the patient's cooperation, and the above-mentioned factors that might affect the results. While EEG provides valuable real-time data on brain function, it might not catch intermittent or infrequent abnormalities in brain activity if they don't occur during the test. Therefore, it's often used alongside other diagnostic methods, like MRI or CT scans, to provide a more complete picture of brain health and accurate diagnosis.


## **Brain and Alcohol**
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6668890/ searched deeper for Alcoholism and Human Electrophysiology.

Interestingly, the article suggests that the observed electrical abnormalities in the brains of alcoholics might not be a result of alcohol consumption per se, but rather a pre-existing condition, possibly serving as a risk marker for alcoholism. Some of these electrical characteristics, such as increased resting beta power and decreased active theta oscillations during cognitive tasks, have also been identified in individuals at high risk for developing alcoholism, even before any exposure to alcohol. Therefore, the text proposes that an inherent imbalance in CNS excitation and inhibition might predispose individuals to alcoholism. This imbalance is suggested to not only contribute to the risk of developing alcoholism but might also offer insights into the neurobiology of craving and relapse in alcoholism

## Dataset

In [1]:
%%shell
wget https://archive.ics.uci.edu/static/public/121/eeg+database.zip
unzip -q eeg+database.zip
gunzip -k eeg_full/*.gz
for file in eeg_full/*.tar; do tar -xf $file -C eeg_full; done
gunzip -k eeg_full/*/*.gz
rm eeg_full/*.tar.gz eeg_full/*.tar eeg_full/*/*.gz
mkdir train
mkdir test

--2023-10-12 15:23:48--  https://archive.ics.uci.edu/static/public/121/eeg+database.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘eeg+database.zip’

eeg+database.zip        [            <=>     ] 762.44M  49.9MB/s    in 17s     

2023-10-12 15:24:05 (44.9 MB/s) - ‘eeg+database.zip’ saved [799481741]



In [2]:
!pip install -q -U mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 42.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import mne
import os

from tqdm import tqdm




In [17]:
def extract_raw_data(file):
  with open(file) as f:
    lines = f.readlines()

    data = []
    channel_names = []
    is_Alcolic = lines[0][5] == 'a'
    id_patient = int(lines[0][6:13])


    if(len(lines)<=3):
      return None,None,None

    l3 = lines[3].split()
    obj = l3[1]
    trial = int(lines[4].split()[-1])


    for line in lines[4:]:
      line_split = line.split()

      if (not line.startswith('#')):
        values = line_split[-1]
        data.append(values)
      else :
        ch_name = line_split[1]
        channel_names.append(ch_name)

    data = np.array(data, dtype="float").reshape((64, -1)) * 1e-6
    info = {"his_id" : id_patient,  "is_Alcolic" : is_Alcolic,'id': id_patient,
            "trial": trial, "obj": obj}
    return data, channel_names, info


In [14]:
! cat /content/eeg_full/co2c1000367/co2c1000367.rd.089

# co2c1000367.rd
# 120 trials, 64 chans, 416 samples 368 post_stim samples
# 3.906000 msecs uV


In [16]:
extract_raw_data('/content/eeg_full/co2c1000367/co2c1000367.rd.089')

/content/eeg_full/co2c1000367/co2c1000367.rd.089
/content/eeg_full/co2c1000367/co2c1000367.rd.089 seems to be emmpty


(None, None, None)

In [18]:
def save_raw_files(file, save=False, train=True):
  file_name = file.split("/")[-1].replace(".", "_") + "_eeg"

  data, ch_names, info_patient = extract_raw_data(file)
  if(data is None):
    return None
  info = mne.create_info(ch_names=ch_names, sfreq=256, ch_types='eeg')

  raw = mne.io.RawArray(data=data, info=info, verbose=False)
  raw.info['subject_info'] = info_patient


  if (save):
    dir = "train" if train else "test"
    raw.save(f"{dir}/{file_name}.fif", overwrite=True, verbose=False)
  return raw



In [19]:
for dir_name, subdirs, files in tqdm(list(os.walk('/content/'))):
    for file_name in files[1:]:
      if ((".rd.") in file_name):
        current_file = os.path.join(dir_name, file_name)
        #train = "TRAIN" in current_file
        save_raw_files(current_file, save=True)


 55%|█████▍    | 72/131 [04:10<03:45,  3.83s/it]

/content/eeg_full/co2c1000367/co2c1000367.rd.089 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.005 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.113 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.023 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.042 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.053 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.090 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.114 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.004 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.006 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.117 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.105 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.037 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.029 seems to be emmpty
/content/eeg_full/co2c1000367/co2c1000367.rd.065

 56%|█████▌    | 73/131 [04:14<03:44,  3.87s/it]

/content/eeg_full/co2c1000367/co2c1000367.rd.054 seems to be emmpty


100%|██████████| 131/131 [07:41<00:00,  3.52s/it]


Create two directroies which which will contain data as csv.

In [22]:
! [ -e train_csv ] || mkdir train_csv
! [ -e test_csv ] || mkdir test_csv

Generate a dataframe from an eeg file.

In [23]:

def generate_df(file,save=False,train=True,verbose=False,frequency = 256):
  df = pd.DataFrame()
  data, channel_names, info = extract_raw_data(file)
  file_name = file.split("/")[-1]
  for time_series,channel in zip(data, channel_names):
    time = np.arange(0,time_series.shape[-1])*1/time_series.shape[-1]
    cur_df = pd.DataFrame({'value':time_series,'channel':channel,**info,'time':time})
    if(df is None):
      df = cur_df
    else:
      df = pd.concat([df,cur_df],ignore_index=True)
  if(save):
    dir = "train" if train else "test"
    saving_path = f'{dir}_csv/{file_name}.csv'
    df.to_csv(saving_path)
  return df



In [ ]:
df = generate_df('/content/SMNI_CMI_TRAIN/co2c0000338/co2c0000338.rd.014')
df.head()

In [ ]:
df[df['channel'] == 'Y'].plot(x='time',y='value')

In [ ]:
def generated_csv(content_dir:str='/content/',train=True):
  for dir_name, subdirs, files in tqdm(list(os.walk(content_dir))):

    for file_name in (files):
      current_file = os.path.join(dir_name, file_name)
      regex = r'.+\.rd\.\d+$'

      if (re.match(regex,file_name)):
        generate_df(current_file,save=True,train=train)


In [ ]:
for data_dir,train in zip(['SMNI_CMI_TRAIN','SMNI_CMI_TEST'],[True,False]):
  generated_csv(data_dir,train)

In [ ]:
content = list(os.walk('train'))
raw = mne.io.read_raw(f"{content[0][0]}/{content[0][2][0]}")
raw.plot()

In [ ]:
raw.get_data()

In [24]:
from scipy.stats import skew, kurtosis

def extract_features(data):
  features_names = []
  features = []
  for i, channel_data in enumerate(data):
      mean = np.mean(channel_data)
      var = np.var(channel_data)
      skewness = skew(channel_data)
      kurt = kurtosis(channel_data)
      channel_features = [mean, var, skewness, kurt]
      channel_names = ['mean', 'var', 'skewness', 'kurt']

      features.extend(channel_features)
      features_names.extend(list(map(lambda x : x + f"_{i}", channel_names)))

  return features, features_names


In [26]:
def create_df(dir):
  df_data = []
  for dir, subdir, files in tqdm(list(os.walk(dir))):
    for f in files[1:]:
      data, ch_names, info= extract_raw_data(f"/content/{dir}/{f}")
      features, features_names = extract_features(data)
      features.extend([info['obj'], info['is_Alcolic']])
      df_data.append(features)
  features_names.extend(['obj', 'alcoholic'])
  return pd.DataFrame(df_data, columns=features_names)

df_train = create_df("eeg_full")
#df_test = create_df("SMNI_CMI_TEST")
df_train.head()

 35%|███▍      | 43/123 [07:30<13:58, 10.48s/it]


KeyboardInterrupt: ignored

In [21]:
import torch
from torch.utils.data import Dataset

In [ ]:
def create_ds(dir):
  df_data = []
  for dir, subdir, files in tqdm(list(os.walk(dir))):
    for f in files[1:]:
      data, ch_names, info= extract_raw_data(f"/content/{dir}/{f}")
      features, features_names = extract_features(data)
      features.extend([info['obj'], info['is_Alcolic']])
      df_data.append(features)
  features_names.extend(['obj', 'alcoholic'])

In [ ]:
df_train.shape, df_test.shape

In [ ]:
def preprocess(df):
  dummies = pd.get_dummies(df['obj'])
  df = df.drop(columns="obj")
  df = pd.concat([df, dummies], axis=1)
  df['alcoholic'] = df['alcoholic'].replace({True: 1, False: 0}).astype(int)

  return df.dropna()

df_train = preprocess(df_train)
df_test = preprocess(df_test)
df_train.head()

In [ ]:
plt.hist(df_train['alcoholic'], alpha=0.7, edgecolor="k")
plt.title("Dataframe histogramme")
plt.ylabel("Count")
plt.xticks([0, 1], ['Control', 'Alcoholic'])
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(df_train)
scaled = scaler.transform(df_train)
scaled_test = scaler.transform(df_test )

df_train_scaled = pd.DataFrame(scaled, columns=df_train.columns)
df_test_scaled = pd.DataFrame(scaled, columns=df_train.columns)
df_train_scaled.head()

In [ ]:
from sklearn.svm import SVR


X_train, y_train = df_train_scaled.loc[:, df_train_scaled.columns != 'alcoholic'], df_train_scaled['alcoholic']
X_test, y_test = df_test_scaled.loc[:, df_test_scaled.columns != 'alcoholic'], df_test_scaled['alcoholic']
regr = SVR(C=1.0, epsilon=0.2)
regr.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(regr.predict(X_test).astype(int), y_test.astype(int))